# Vigilant Asset Allocation (VAA)

Vigilant Asset Allocation (VAA) is introduced by Wouter J. Keller and Jan Willem Keuning in research paper "Breadth Momentum and Vigilant Asset Allocation (VAA): Winning More by Losing Less" (https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3002624)


In [11]:
import pandas as pd
import src.fmp as fmp
import datetime as dt

## Portfolio Assets

In [12]:
offensive = ['SPY', 'VEA', 'VWO', 'AGG']
defensive = ['SHY', 'IEF', 'LQD']

## Upgrade version of dual momentum

1. We buy the strongest asset in terms of momentum in offensive assets.
2. If the most recent return of any of the offensive assets is negative, we go to defensive asset.

## How to compute recent returns and weights

recent_return = 12 * (recent 1M return) + 4 * (recent 4M return) + 2 * (recent 6M return) + 1 * (recent 12M return)

In [29]:
offensive_prices = pd.DataFrame()
monthly = pd.DataFrame()

for symbol in offensive:
    data = fmp.get_daily_prices(symbol)
    offensive_prices[symbol] = data['Close']

print(offensive_prices.head(10))

for i in range(0,len(offensive_prices.index)-1):
    currMonth = dt.datetime.strptime(offensive_prices.index[i], '%Y-%m-%d').month
    nextMonth = dt.datetime.strptime(offensive_prices.index[i+1], '%Y-%m-%d').month
    
    if currMonth != nextMonth:
        monthly = monthly.append(offensive_prices.loc[offensive_prices.index[i]])
    
monthly

                   SPY        VEA        VWO         AGG
Date                                                    
2016-09-26  214.240005  37.110001  37.230000  112.370003
2016-09-27  215.570007  37.290001  37.619999  112.510002
2016-09-28  216.639999  37.619999  37.990002  112.540001
2016-09-29  214.679993  37.160000  37.360001  112.540001
2016-09-30  216.300003  37.410000  37.630001  112.419998
2016-10-03  215.779999  37.340000  37.880001  112.089996
2016-10-04  214.679993  37.230000  37.500000  111.709999
2016-10-05  215.630005  37.430000  38.070000  111.559998
2016-10-06  215.779999  37.230000  38.070000  111.559998
2016-10-07  215.039993  36.990002  37.939999  111.580002


,AGG,SPY,VEA,VWO
2016-09-30,112.419998,216.300003,37.410000,37.630001
2016-10-31,111.300003,212.550003,36.509998,37.750000
2016-11-30,108.239998,220.380005,35.959999,36.220001
2016-12-30,108.059998,223.529999,36.540001,35.779999
2017-01-31,108.290001,227.529999,37.880001,37.840000
2017-02-28,108.769997,236.470001,38.279999,38.689999
2017-03-31,108.489998,235.740005,39.299999,39.720001
2017-04-28,109.250000,238.080002,40.169998,40.340000
2017-05-31,109.760002,241.440002,41.540001,40.740002
2017-06-30,109.510002,241.800003,41.320000,40.830002


## Offensive Assets Momentum

In [4]:
mom = {'1M': [], '3M': [], '6M': [], '12M': []}

for symbol in offensive:
    mom['1M'].append(fmp.calculate_hist_momentum(symbol, 30))
    mom['3M'].append(fmp.calculate_hist_momentum(symbol, 90))
    mom['6M'].append(fmp.calculate_hist_momentum(symbol, 180))
    mom['12M'].append(fmp.calculate_hist_momentum(symbol, 252))

mom

{'1M': [-0.004336006829772431,
  -0.0026631158455392915,
  -0.010487473295785572,
  0.0005192557334487432],
 '3M': [0.07583628161462216,
  0.015888393722146878,
  -0.031000380372765226,
  0.013589356717330527],
 '6M': [0.1690318991975959,
  0.07703368940016433,
  -0.009140373751084098,
  -0.013734865946639916],
 '12M': [0.3699536321483771,
  0.3026086956521739,
  0.20849146110056943,
  -0.021000898331811273]}

In [5]:
offensive_momentum = pd.DataFrame(mom, index=offensive)
offensive_momentum['Score'] = 12 * offensive_momentum['1M'] + 4 * offensive_momentum['3M'] + 2 * offensive_momentum['6M'] + 1 * offensive_momentum['12M']
offensive_momentum

,1M,3M,6M,12M,Score
SPY,-0.004336,0.075836,0.169032,0.369954,0.959330
VEA,-0.002663,0.015888,0.077034,0.302609,0.488272
VWO,-0.010487,-0.031000,-0.009140,0.208491,-0.059640
AGG,0.000519,0.013589,-0.013735,-0.021001,0.012118


## Defensive Assets Momentum

In [6]:
mom = {'1M': [], '3M': [], '6M': [], '12M': []}

for symbol in defensive:
    mom['1M'].append(fmp.calculate_hist_momentum(symbol, 30))
    mom['3M'].append(fmp.calculate_hist_momentum(symbol, 90))
    mom['6M'].append(fmp.calculate_hist_momentum(symbol, 180))
    mom['12M'].append(fmp.calculate_hist_momentum(symbol, 252))

mom

{'1M': [-0.000696136442742804, -0.004375053615853058, 0.007676255775823529],
 '3M': [-0.0018542241064524762, 0.017713074204550372, 0.036330190848470986],
 '6M': [-0.0026632815810180626, -0.021251492304747904, -0.00478434414261489],
 '12M': [-0.0041623078774958985, -0.048532569719644904, 0.006326280095815099]}

In [7]:
defensive_momentum = pd.DataFrame(mom, index=defensive)
defensive_momentum['Score'] = 12 * defensive_momentum['1M'] + 4 * defensive_momentum['3M'] + 2 * defensive_momentum['6M'] + 1 * defensive_momentum['12M']
defensive_momentum

,1M,3M,6M,12M,Score
SHY,-0.000696,-0.001854,-0.002663,-0.004162,-0.025259
IEF,-0.004375,0.017713,-0.021251,-0.048533,-0.072684
LQD,0.007676,0.036330,-0.004784,0.006326,0.234193
